In [3]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr connect --cluster-id j-104TXHDV2M8Z9 --auth-type None --language python  

Successfully read emr cluster(j-104TXHDV2M8Z9) details
Initiating EMR connection..
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
8,application_1682640635824_0009,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
{"namespace": "sagemaker-analytics", "cluster_id": "j-104TXHDV2M8Z9", "error_message": null, "success": true, "service": "emr", "operation": "connect"}


In [6]:
from pyspark.sql import SparkSession
import pandas as pd

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
pd.set_option('display.max_columns', 100) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows', 1000) #replace n with the numbe

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Configure location of raw parquet file
FILENAME = "s3://sagemaker-us-east-2-850751315356/vedjain-deltalake-2022/lending_club/full-loans-pq/part-00000-bb6c5c3e-bcce-4026-a4cf-9dda63dcf794-c000.snappy.parquet"

# Read loanstats_2012_2017.parquet
data = spark.read.parquet(FILENAME)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
data.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[id: string, member_id: string, loan_amnt: float, funded_amnt: int, funded_amnt_inv: double, term: string, int_rate: string, installment: double, grade: string, sub_grade: string, emp_title: string, emp_length: string, home_ownership: string, annual_inc: float, verification_status: string, loan_status: string, pymnt_plan: string, url: string, desc: string, purpose: string, title: string, zip_code: string, addr_state: string, dti: float, delinq_2yrs: float, earliest_cr_line: string, inq_last_6mths: string, mths_since_last_delinq: int, mths_since_last_record: string, open_acc: int, pub_rec: int, revol_bal: int, revol_util: string, total_acc: float, initial_list_status: string, out_prncp: string, out_prncp_inv: double, total_pymnt: string, total_pymnt_inv: double, total_rec_prncp: double, total_rec_int: double, total_rec_late_fee: double, recoveries: double, collection_recovery_fee: double, last_pymnt_d: string, last_pymnt_amnt: string, next_pymnt_d: string, last_credit_pull_d: s

In [10]:
print(data.limit(5).toPandas())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     id member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  None      None    35000.0        35000          35000.0   36 months   
1  None      None     8000.0         8000           8000.0   36 months   
2  None      None     5000.0         5000           5000.0   36 months   
3  None      None    10000.0        10000          10000.0   36 months   
4  None      None    24000.0        24000          24000.0   36 months   

  int_rate  installment grade sub_grade             emp_title emp_length  \
0   17.27%      1252.56     D        D2                 Owner  10+ years   
1   18.25%       290.23     D        D3              Manager     9 years   
2    6.97%       154.32     A        A3                  None        n/a   
3    9.75%       321.50     B        B3           ELECTRICIAN  10+ years   
4    9.75%       771.60     B        B3  C&C MACHINE OPERATOR    5 years   

  home_ownership  annual_inc verification_status         loan_status  \
0           RENT    107000

In [11]:
data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: float (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: float

#### Dataset details can be found here: https://www.kaggle.com/datasets/wordsforthewise/lending-club

In [ ]:
data.schema.names

### Munge Data - create feature columns

In [12]:
from pyspark.sql.functions import *

print("------------------------------------------------------------------------------------------------")
print("Create bad loan label, this will include charged off, defaulted, and late repayments on loans...")
data = data.filter(data.loan_status.isin(["Default", "Charged Off", "Fully Paid"]))\
                       .withColumn("bad_loan", (~(data.loan_status == "Fully Paid")).cast("string"))

print("------------------------------------------------------------------------------------------------")
print("Turning string interest rate and revoling util columns into numeric columns...")
data = data.withColumn('int_rate', regexp_replace('int_rate', '%', '').cast('float')) \
                       .withColumn('revol_util', regexp_replace('revol_util', '%', '').cast('float')) \
                       .withColumn('issue_year',  substring(data.issue_d, 5, 4).cast('double') ) \
                       .withColumn('earliest_year', substring(data.earliest_cr_line, 5, 4).cast('double'))
data = data.withColumn('credit_length_in_years', (data.issue_year - data.earliest_year))


print("------------------------------------------------------------------------------------------------")
print("Converting emp_length column into numeric...")
data = data.withColumn('emp_length', trim(regexp_replace(data.emp_length, "([ ]*+[a-zA-Z].*)|(n/a)", "") ))
data = data.withColumn('emp_length', trim(regexp_replace(data.emp_length, "< 1", "0") ))
data = data.withColumn('emp_length', trim(regexp_replace(data.emp_length, "10\\+", "10") ).cast('float'))

print("------------------------------------------------------------------------------------------------")
print("Map multiple levels into one factor level for verification_status...")
data = data.withColumn('verification_status', trim(regexp_replace(data.verification_status, 'Source Verified', 'Verified')))

print("------------------------------------------------------------------------------------------------")
print("Calculate the total amount of money earned or lost per loan...")
data = data.withColumn('net', round( data.total_pymnt - data.loan_amnt, 2))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

------------------------------------------------------------------------------------------------
Create bad loan label, this will include charged off, defaulted, and late repayments on loans...
------------------------------------------------------------------------------------------------
Turning string interest rate and revoling util columns into numeric columns...
------------------------------------------------------------------------------------------------
Converting emp_length column into numeric...
------------------------------------------------------------------------------------------------
Map multiple levels into one factor level for verification_status...
------------------------------------------------------------------------------------------------
Calculate the total amount of money earned or lost per loan...

In [13]:
print(data.select("net","verification_status","int_rate", "revol_util", "issue_year", "earliest_year", "bad_loan", "credit_length_in_years", "emp_length").limit(10).toPandas())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

        net verification_status  int_rate  revol_util  issue_year  \
0     98.34        Not Verified      5.32   27.900000      2016.0   
1    263.31            Verified      9.75   19.400000      2016.0   
2    361.74        Not Verified      5.32   23.900000      2016.0   
3   1088.67            Verified     12.99   75.400002      2016.0   
4 -20306.74            Verified     21.18   87.500000      2016.0   
5    330.35            Verified      7.39   59.099998      2016.0   
6 -14837.19            Verified     15.31    7.700000      2016.0   
7   2405.39            Verified     17.27   72.000000      2016.0   
8    632.18            Verified      9.16   28.799999      2016.0   
9   1044.92        Not Verified     15.31   97.199997      2016.0   

   earliest_year bad_loan  credit_length_in_years  emp_length  
0         2000.0    false                    16.0         8.0  
1         2010.0    false                     6.0         1.0  
2         1989.0    false                    27.

### Set Response & Predictor Variables

In [44]:
print("------------------------------------------------------------------------------------------------")
print("Setting variables to predict bad loans")
myY = "bad_loan"
categoricals = ["term", "home_ownership", "purpose", "addr_state",
                "verification_status","application_type"]
numerics = ["loan_amnt","emp_length", "annual_inc","dti",
            "delinq_2yrs","revol_util","total_acc",
            "credit_length_in_years"]
myX = categoricals + numerics

loan_stats = data.select(myX + [myY, "int_rate", "net", "issue_year"])
loan_stats.createOrReplaceTempView("loanStatsView")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

------------------------------------------------------------------------------------------------
Setting variables to predict bad loans

#### You can use SparkSQL queries using %%sql from the notebook and save results to a local DataFrame. This allows for a quick data exploration. The maximum rows returned by default is 2,500. You can set the maximum rows by using the -n argument.

In [15]:
%%sql -o loan_stats
select * from loanStatsView

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

#### You can then access and explore Spark dataframe locally

In [17]:
%%local 
loan_stats.head(10)

Output()

In [29]:
%%local 
loan_stats.columns

Index(['term', 'home_ownership', 'purpose', 'addr_state',
       'verification_status', 'application_type', 'loan_amnt', 'emp_length',
       'annual_inc', 'dti', 'delinq_2yrs', 'revol_util', 'total_acc',
       'credit_length_in_years', 'bad_loan', 'int_rate', 'net', 'issue_year'],
      dtype='object')

In [45]:
# SageMaker XGBoost has the convention of label in the first column
loan_stats = loan_stats.toPandas()
label = loan_stats.pop('bad_loan')
loan_stats.insert(0, 'bad_loan', label)
  
loan_stats[categoricals] = loan_stats[categoricals].apply(lambda x: x.astype("category").cat.codes)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Create Train / Validation Split for SageMaker XGBoost Model

In [47]:
%%local

# # Split the downloaded loan_stats into train/test loan_statsframes
train, test = np.split(loan_stats.sample(frac=1), [int(0.8 * len(loan_stats))])

# # requires PyArrow installed
train.to_parquet("lending_club_train.parquet")
test.to_parquet("lending_club_test.parquet")

In [ ]:
%%local 

train = loan_stats.filter(loan_stats.issue_year <= 2015).cache()
valid = loan_stats.filter(loan_stats.issue_year > 2015).cache()

print("training data size = %d | validation data size = %d" % (train.count(), valid.count()))

In [6]:
import io
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = sagemaker.Session().default_bucket()
prefix = "vedjain-deltalake-2022/lending_club/sagemaker-xg-boost"
# customize to your bucket where you have would like to store the data
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 'sagemaker'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'sagemaker'



In [27]:
%%local
import boto3
import numpy as np
import pandas as pd
from sklearn.datasets import load_svmlight_file

s3 = boto3.client("s3")
# Download the dataset and load into a pandas dataframe
FILE_NAME = "abalone.csv"
s3.download_file("sagemaker-sample-files", f"datasets/tabular/uci_abalone/abalone.csv", FILE_NAME)
feature_names = [
    "Sex",
    "Length",
    "Diameter",
    "Height",
    "Whole weight",
    "Shucked weight",
    "Viscera weight",
    "Shell weight",
    "Rings",
]
data = pd.read_csv(FILE_NAME, header=None, names=feature_names)
data

Output()

In [28]:
%%local
# SageMaker XGBoost has the convention of label in the first column
data = data[feature_names[-1:] + feature_names[:-1]]
data["Sex"] = data["Sex"].astype("category").cat.codes
data

Output()